## Introduction to Specviz

#### Learning Goals

- Learn to load and display a spectrum in Specviz
- Become familiar with the tools attached to the spectrum viewer
- Become familiar with the data analysis plugins
- Learn how to select a region of interest/subset of the spectrum
- Learn how to overlay spectral lines on the displayed spectrum
- Learn how to extract spectra from the Specviz app for further analysis in the notebook

#### Historical Note

Previously, Specviz was one of three separate packages (Specviz, Cubeviz, and Mosviz) for visual data analysis. These packages have now been unified into separate configurations of a single project called Jdaviz and share much of the same codebase. The main relevance this has to you is that you should _not_ try to install Specviz via `pip install specviz`, as this will give you the old, no-longer-maintained package. Instead, always use `pip install jdaviz` to get Specviz and the other configurations.

#### Further Resources

The Specviz documentation can be found at https://jdaviz.readthedocs.io/en/latest/specviz/index.html. 

In [ ]:
# This ensures that our notebook is using the full width of the browser

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the packages and modules we'll need

from jdaviz import SpecViz

from astropy.utils.data import download_file
from astropy.table import QTable
import astropy.units as u

### Running the app

To initialize the Specviz app, we import the `Specviz` class, initialize an instance of it, and invoke the `app` property to display the app. We also import a few more things here that will be useful later.

In [ ]:
specviz = SpecViz()
specviz.app

### Loading data into the app

Loading data is as simple as calling the `load_spectrum` method of the `Specviz` class. Here, we download a spectrum (extracted from a simulated observation of a galaxy through the NIRSpec IFU) and load it into the app, making use of the optional `data_label` argument to give the data a sensible label. Note that `load_spectrum` can also be given `specutils.Spectrum1D` objects as input. 

In [ ]:
# First, download the file we'll be using. 
# This 1D spectrum is extracted from a simulated observation of a galaxy through the NIRSpec IFU.
fn = download_file('https://stsci.box.com/shared/static/b22b1fzhimtdqfp8597m4bg67kovvauu.fits', cache=True)

# Now load it into the app
specviz.load_spectrum(fn, data_label="Demo")

### Selecting a region of interest

First, note that before selecting any subsets, the subset selection button in the app-level toolbar reads "No selection (create new)". This indicates that using the x-range selection tool will create a new subset, rather than editing an existing subset. To the right of that button you should see a small circle - clicking that circle shows that there are options for what the x-range selection will do. Since we're creating a new subset, the default ("replace") is fine.

Follow these steps to select a new region:
1. Click the "hammer and screwdriver" icon in the top left of the viewer. 
2. Use any of the pan/zoom tools (the leftmost three icons in the expanded tool tray) to zoom into the feature of interest. We'll investigate the emission feature near 1.25 microns. Zooming works by scrolling with the mouse wheel (see "Sidebar: zooming from the notebook" below if you can't scroll to zoom).
3. Click the "x-range" selection tool in the expanded tool tray.
4. Click and drag on the viewer to select a region around the feature. Go ahead and include some continuum on either side.
5. Click the "x-range" button in the expanded tool tray to deselect the tool.

You should now see the label "Subset 1" with a red triangle in the app-level toolbar where it previously said "No selection (create new)". The red triangle indicates that the region of the spectrum you selected in the viewer should be colored red to indicate it is in this subset.

#### Sidebar: zooming from the notebook

If you don't have the capability to zoom via scrolling with a zoom tool selected, you can set the limits of the x and y axes manually from the notebook by calling the `specviz.x_limits` and `specviz.y_limits` methods.

In [ ]:
specviz.x_limits(1.2, 1.3)

specviz.y_limits('auto', 3)

### Creating and fitting models

Now that we've created let the app know that we're interested in the region around the emission line by creating a subset, we can use the Model Fitting plugin to do some analysis by fitting the line. Follow these steps:

1. Click the "lego block" icon in the top right of the app to open the plugin menu, then click the `Model Fitting` line to expand that plugin.
2. Click the `Data` dropdown menu and select "Subset 1".
3. Click the `Model` dropdown and select "Constant", then give the component model a label in the `Model ID` field immediately to the right (I'll label it "C"). Click `ADD MODEL` and note that an expandable menu appears for model `C` under "Model Parameters".
4. Repeat step 3, but this time add a Gaussian component.
5. Expand the model parameter menu for the Gaussian model and give it some reasonable initial guesses for the parameters. I'll use `amplitude=1.5`, `stddev=0.0005`, `mean=1.252`.
6. In the input field in the "Model Equation Editor" section, type "C+G" (or whatever the two labels you gave to your component models were). This tells the fitter how to combine the component models you've initialized.
7. Update the label for the resulting model, if desired. The default is "Model".
8. Click "FIT". You should see a the fit model appear in the viewer, overlayed on the original spectrum. 

### Adding line lists via the notebook

Line lists are expected to be input as an astropy `QTable`, with at minimum "linename" and "rest" fields. You can optionally specify a "color" column if you want each line to be a different color. 

The first step is to load the line list into the specviz instance using `specviz.load_line_list(lines)`. A line list as described as above is required, and there is an optional `replace` parameter that, if set to True, replaces any existing lines with those input. The default behavior is to add newly loaded lines to the existing list.

Note that the line list functionality is all actually happening under the hood in the spectrum viewer (and thus could also be used in e.g. Cubeviz); the Specviz methods are convenience wrappers. 

In [ ]:
lt = QTable()
lt['linename'] = ['Test1', 'Test2']
lt['rest'] = [0.5007, 0.6563]*u.um
lt['redshift'] = u.Quantity(0.046)
lt['listname'] = "Test"

specviz.load_line_list(lt)

Displaying the lines on the plot is as simple as calling the `plot_spectral_lines` function:

In [ ]:
specviz.plot_spectral_lines()

And one can erase all spectral lines from the plot using `erase_spectral_lines`:

In [ ]:
specviz.erase_spectral_lines()

### Using the Line Lists plugin

We can also add and remove lines via the Line Lists plugin, including a set of predefined line lists that can be loaded and plotted by the user. These are viewable in the plugin under the "Available Line Lists" dropdown. Once a list is added, you can open the list menu and choose to display and hide either individual lines from the list or all the lines in the list. Note that showing more than a dozen or so lines at once can start impacting the performance of the redshift slider.

### Retrieving data

The data from the app can be retrieved into the notebook using the `get_spectra` method of the `Specviz` class. Retrieve the data using that method below. Note that the returned objects are `specutils.Spectrum1D` instances, which you learned about in the previous session.

In [ ]:
spectra = specviz.get_spectra()
print(spectra)

## Exercises


1. Use the subset selection tool to add the next feature to the left of the one we already fit to Subset 1. Then use the Model Fitting plugin to fit multiple gaussians at once, one for each emission line. Use the `get_models` method of the `specviz` object to retrieve the fitted models into the notebook.

    **Reminder**: The "Model ID" of any component model you initialize in the Model Fitting plugin needs to be used in the `Model Equation Editor` section of the plugin before pressing the `Fit` button.


2. Create a subset that includes only continuum, then use the Model Fitting plugin to fit a model to that subset. Using `specviz.get_spectra()`, retrieve the data from the app and subtract the continuum fit from the spectrum. Load the subctracted spectrum back into the app.

3. Use the Unit Conversion plugin to change the units of the displayed spectrum. Then use the data selection menu to switch back to the original spectrum.

### Solutions

#### Exercise 1

I created three component models: a Constant (C), Gaussian (G1), and another Gaussian (G2). Good initial first guesses were:

C: amplitude = 0.7
G1: amplitude = 1, stddev = 0.0005, mean = 1.253
G2: amplitude = 0.5, stddev = 0.0007, mean = 1.24

Note that you likely already have the fitted parameters in the UI for C and G1 from earlier, so you could leave those unchanged and simply add G2. Make sure your "Model Equation" field reads "C+G1+G2" before clicking `Fit`. The fitted models can be retrieved in the notebook as below:

In [ ]:
fit_models = specviz.get_models()
print(fit_models)

#### Exercise 2

I chose "No selection (create new)" in the subset menu, selected the "Add" region selection behavior, and then selected three regions of continuum: 1.05-1.2, 1.3-1.6, and 1.7 to the end of the spectrum. If you have only created and edited until this point Subset 1, this should result in the creation of Subset 2 in green. I also deselected the previous fitted model ("Model") from the viewer data selection dropdown menu to hide it. 

I then went into the Model Fitting plugin, deleted the previous model componenents, and selected "Subset 2" in the data dropdown menu. Be sure to give the model a new label in the Model Label field - I chose "Continuum". I chose a Polynomial component model, labeled it "P3" and increased the order to 3, and left the parameters with the default estimates of 0, since pressing `Fit` resulted in a good fit even with the default estimates. Retrieving the spectrum and continuum fit and subtracting the two is shown below:

In [ ]:
spectra = specviz.get_spectra()

# I named the continuum model I created in the Model Fitting plugin "Continuum"
# Note that you could do normalize the spectrum instead of subtracting the continuum by dividing instead of subtracting.
normalized = spectra["Demo"] - spectra["Continuum"]

specviz.load_spectrum(normalized, data_label = "Subtracted Spectrum")

#### Exercise 3

In the Units Conversion plugin I selected "Hertz" for "New Spectral Axis Unit and "W / (Hz m2)" for "New Flux Unit". Clicking the `Apply` button should display the converted spectrum automatically. To switch back to the original units, click the hammer and screwdriver icon in the top left of the viewer (if you don't have the oolbar expanded already), then the gear icon in the expanded toolbar, then uncheck the converted copy of the data in the data selection menu and re-check the original spectrum.